## Generating a Table of Global Fluxes

This notebook reads in the CESM1 historical run (for CMIP5),
the ensemble of 11 CESM2 historical runs (for CMIP6),
and also the four SSP CESM2 ensembles (for CMIP6).
A table is generated containing values listed [issue #6](https://github.com/marbl-ecosys/cesm2-marbl/issues/6)


> * Net primary production (PgC/yr) (`photoC_TOT_zint`)
> * Diatom primary production (%)   (`photoC_diat_zint`)
> * Sinking POC at 100 m (PgC/yr)   (`POC_FLUX_100m`)
> * Sinking CaCO3 at 100 m (PgC/yr) (`CaCO3_FLUX_100m`)
> * Rain ratio (CaCO3/POC) 100 m    (ratio of two above)
> * Nitrogen fixation (TgN/yr)      (`diaz_Nfix`)
> * Nitrogen deposition (TgN/yr)    (`NOx_FLUX` + `NHy_FLUX`)
> * Denitrification (TgN/yr)        (`DENITRIF`)
> * N cycle imbalance = deposition + fixation - denitrification (TgN/yr) # deposition = N* [see Kristen's notebook -- Biological Diagnostics?]
> * Air–sea CO2 flux (PgC yr21)     (`FG_CO2`)
> * Mean ocean oxygen (uM = umol/L = mmol/m^3)    (`O2`)
> * Volume where O2 <80 mmol/m^3 (10^15 m^3) # based on others
> * Volume where O2 <60 mmol/m^3 (10^15 m^3) # based on others
> * Volume where O2 <5 mmol/m^3 (10^15 m^3)  # based on others

Values will be computed one at a time, due to an issue with `xr.merge` and trying to read multiple variables at once.

### This notebook uses several python packages

The watermark package shows the version number used to help others recreate this environment.

In [1]:
%matplotlib inline
import os

import cftime

import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean

import cartopy
import cartopy.crs as ccrs

import esmlab

import intake
import intake_esm
import ncar_jobqueue
from dask.distributed import Client
from pint import UnitRegistry
units = UnitRegistry()

%load_ext watermark
%watermark -a "Mike Levy" -d -iv -m -g -h

numpy         1.17.3
esmlab        2019.4.27.post55
cartopy       0.17.0
intake        0.5.3
pandas        0.25.3
intake_esm    2019.10.15.post40
matplotlib    3.1.2
xarray        0.14.0
ncar_jobqueue 2019.10.16.1
cmocean       2.0
cftime        1.0.3.4
Mike Levy 2019-11-20 

compiler   : GCC 7.3.0
system     : Linux
release    : 3.10.0-693.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : casper02
Git hash   : 04204b5b80a01626715ad5d20449aa875f5b3093


#### Spin up a dask cluster

Some of these computations take a while

In [2]:
cluster = ncar_jobqueue.NCARCluster(project='P93300606')
client = Client(cluster)
client

Client Scheduler: tcp://128.117.181.208:46243 Dashboard: https://jupyterhub.ucar.edu/dav/user/mlevy/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
cluster.scale(4)

### Read the intake_esm datastores

The `intake_esm` package is used to help identify which files belong in each experiment.
The `get_var_from_catalogs()` function is a wrapper to read specific files.

In [4]:
catalogs = dict()
catalogs['cesm2'] = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/campaign-cesm2-cmip6-timeseries.json')

#cesm1 = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip5_NOT_CMORIZED.json')
catalogs['cesm1'] = intake.open_esm_datastore('/glade/work/mlevy/intake-esm-collection/json/glade-cesm1-cmip5-timeseries.json')

In [5]:
# NOTE: 1991-01-01 0:00:00 is the time stamp on the Dec 1990 monthly average
#       So slice("1991", "2001") would actually return Dec 1990 - Nov 2000
#       Specifying a day mid-month gets us to Jan 1991 - Dec 2000
#       (this can be verified by looking at time bounds)
time_slices_hist = slice("1990-01-15", "2000-01-15")
time_slices_SSP = slice("2090-01-15", "2100-01-15")
time_slices = dict()

time_slices['cesm1_PI'] = slice(290*12, 300*12) # cfunits doesn't years too far in past; this is 291-01-15 - 301-01-15
                                                # (chosen to correspond with 1990 in historical run, which branched from 151)
time_slices['cesm1_hist'] = time_slices_hist
time_slices['cesm1_hist_esm'] = time_slices_hist
time_slices['cesm2_hist'] = time_slices_hist
time_slices['cesm2_SSP1-2.6'] = time_slices_SSP
time_slices['cesm2_SSP2-4.5'] = time_slices_SSP
time_slices['cesm2_SSP3-7.0'] = time_slices_SSP
time_slices['cesm2_SSP5-8.5'] = time_slices_SSP

def get_var_from_catalogs(catalogs, variable, experiments):
    print(f'Reading {variable} from catalog...\n')
    if type(experiments) != list:
        experiments = [experiments]

    datasets = dict()
    experiment_dict = {'cesm1_PI' : ('cesm1', 'piControl'),
                       'cesm1_hist' : ('cesm1', 'historical'),
                       'cesm1_hist_esm' : ('cesm1', 'esm-hist'),
                       'cesm2_hist' : ('cesm2', 'historical'),
                       'cesm2_SSP1-2.6' : ('cesm2', 'SSP1-2.6'),
                       'cesm2_SSP2-4.5' : ('cesm2', 'SSP2-4.5'),
                       'cesm2_SSP3-7.0' : ('cesm2', 'SSP3-7.0'),
                       'cesm2_SSP5-8.5' : ('cesm2', 'SSP5-8.5')
                      }
    
    dq = dict()
    # CESM1 is historical only, CESM2 also has SSPs
    # Note some variable rename shenanigans to account for changes between CESM1 and CESM2
    if variable == 'CaCO3_FLUX_100m':
        cesm1_var = 'CaCO3_FLUX_IN'
    elif variable == 'POC_FLUX_100m':
        cesm1_var = 'POC_FLUX_IN'
    elif variable == 'photoC_diat_zint':
        cesm1_var = 'photoC_diat'
    elif variable == 'photoC_TOT_zint':
        cesm1_var = 'photoC_diaz' # FIXME: need a way to sum photoC_sp, photoC_diat, and photoC_diaz
    else:
        cesm1_var = variable
    dq['cesm1'] = catalogs['cesm1'].search(experiment=['historical', 'esm-hist', 'piControl'], variable=cesm1_var).to_dataset_dict(cdf_kwargs={'chunks':{'time': 60}})
    dq['cesm2'] = catalogs['cesm2'].search(experiment=['historical', 'SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5'], variable=variable).to_dataset_dict(cdf_kwargs={'chunks':{'time': 48}})
    print('\n----\n')

    # Define datasets
    for exp in experiments:
        datasets[exp] = dq[experiment_dict[exp][0]][f'ocn.{experiment_dict[exp][1]}.pop.h']

    keep_vars = ['z_t', 'z_t_150m', 'dz', 'TAREA', 'TLONG', 'TLAT', 'time', 'time_bound', 'member_id', 'ctrl_member_id'] + [variable, cesm1_var]
    for exp in datasets:
        if exp == 'cesm1_PI':
            # Need isel instead of sel since PI slice is in index space rather than years
            datasets[exp] = datasets[exp].drop([v for v in datasets[exp].variables if v not in keep_vars]).isel(time=time_slices[exp])
        else:
            datasets[exp] = datasets[exp].drop([v for v in datasets[exp].variables if v not in keep_vars]).sel(time=time_slices[exp])
        if variable not in datasets[exp]:
            datasets[exp] = datasets[exp].rename({cesm1_var : variable})
            if variable in ['POC_FLUX_100m', 'CaCO3_FLUX_100m']:
                datasets[exp] = datasets[exp].isel(z_t=10)  # 100m is top of 11th level, or z_t = 10 counting from 0
        if (cesm1_var != variable) and (cesm1_var in datasets[exp]):
            datasets[exp] = datasets[exp].drop(cesm1_var)

    return(datasets)

def add_o2_below_thres(datasets, thres):
    new_datasets = datasets.copy()
    varname = f'O2_under_{thres}uM'
    new_datasets[varname] = dict()
    for exp in new_datasets['O2']:
        new_datasets[varname][exp] = new_datasets['O2'][exp].rename({'O2' : varname})
        new_datasets[varname][exp][varname] = xr.where(new_datasets['O2'][exp]['O2'] < thres, 1, 0)
        new_datasets[varname][exp][varname].attrs['units'] = ''
    return(new_datasets)

In [6]:
%%time

experiments = ['cesm1_PI',
               #'cesm1_hist',
               'cesm1_hist_esm',
               #'cesm2_hist',
               'cesm2_SSP5-8.5'
              ]

# Set up units
integral_units = dict()
integral_units['area'] = dict()
integral_units['volume'] = dict()

# Read in any 3D variable to get dataset containing TAREA and z_t
# And set up a DataArray to compute total volume
tmp_var = 'DENITRIF'
tmp_data = get_var_from_catalogs(catalogs, tmp_var, experiments)

total_volume = dict()
for exp in tmp_data:
    integral_units['area'][exp] = units[tmp_data[exp]['TAREA'].attrs['units']]
    integral_units['volume'][exp] = integral_units['area'][exp] * units[tmp_data[exp]['dz'].attrs['units']]

    # Compute total volume of ocean
    # Sum wgt over active ocean cells
    tmp_data2 = tmp_data[exp][tmp_var].isel(time=0, member_id=0).copy()
    tmp_data2.data = np.logical_not(np.isnan(tmp_data[exp][tmp_var].isel(time=0, member_id=0).data))
    total_volume[exp] = (tmp_data2 * tmp_data[exp]['TAREA'].isel(time=0) * tmp_data[exp]['dz'].isel(time=0)).sum().values
    print(f'Ocean volume in {exp}: {(total_volume[exp] * integral_units["volume"][exp]).to("L")}')

vol_exp = experiments[0]
# Estimate total volume in ocean
# (surface of earth is 71% water, avg depth is 3.7 km)
est_depth = 4*np.pi*0.71*((6371.22*units['km'])**2)*(3.7*units['km'])
print(f'Estimated ocean volume: {est_depth.to("L")}')

Reading DENITRIF from catalog...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 3 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)

----

Ocean volume in cesm1_PI: 1.3251402862315105e+21 liter
Ocean volume in cesm1_hist_esm: 1.3251402862315105e+21 liter
Ocean volume in cesm2_SSP5-8.5: 1.325121430696603e+21 liter
Estimated ocean volume: 1.3400319094588907e+21 liter
CPU times: user 2.21 s, sys: 361 ms, total: 2.57 s
Wall time: 1min 19s


### Individual Table Computations

In this section, we compute each of the requested values for each dataset

#### Net primary production (PgC/yr)

CESM1 doesn't have `photoC_TOT_zint`

#### Diatom primary production (%)

CESM1 doesn't have `photoC_diat_zint`

#### Sinking POC at 100 m (PgC/yr)

CESM1 doesn't have `POC_FLUX_100m`

#### Sinking CaCO3 at 100 m (PgC/yr)

CESM1 doesn't have `CaCO3_FLUX_100m`

#### Rain ratio (CaCO3/POC) 100 m

Missing necessary vars to compute

#### Nitrogen deposition (TgN/yr)

#### Denitrification (TgN/yr)

In [7]:
%%time

all_data = dict()

# Process for updating intake-esm catalog
#       1. download all data from HPSS via get_ocn_cmip5_files.sh
#       2. rm /glade/u/home/mlevy/.intake_esm/collections/CESM1-CMIP5.nc
#       3. regenerate it via Anderson's legacy intake-esm
#       4. re-run build intake collections notebook
#       5. commit change to .csv.gz in /glade/work/mlevy/intake-esm-collection/csv.gz/
# NOTE: steps 2-5 can be done with notebooks/intake-esm-collection-defs/rebuild.sh

vars = ['photoC_TOT_zint', 'photoC_diat_zint',
        'POC_FLUX_100m', 'CaCO3_FLUX_100m',
        'diaz_Nfix', 'NOx_FLUX', 'NHy_FLUX', 'DENITRIF',
        'FG_CO2', 'O2']
for var in vars:
    all_data[var] = get_var_from_catalogs(catalogs, var, experiments)

Reading photoC_TOT_zint from catalog...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 3 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)

----

Reading photoC_diat_zint from catalog...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 3 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)

----

Reading POC_FLUX_100m from catalog...

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 3 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'

--> There will be 5 group(s)

----

Reading CaC

In [8]:
# Add variables for computing volume where O2 is below a threshold
if 'O2' in vars:
    all_data = add_o2_below_thres(all_data, 5)
    all_data = add_o2_below_thres(all_data, 20)
    all_data = add_o2_below_thres(all_data, 60)
    all_data = add_o2_below_thres(all_data, 80)

In [9]:
# Verify time bounds for each experiment
for exp in all_data[vars[0]]:
    bounds = list(all_data[vars[0]][exp].time_bound.values[ind] for ind in [(0,0), (-1,1)])
    print(f'Experiment: {exp}\nBounds\n----\n{bounds}\n\n')

Experiment: cesm1_PI
Bounds
----
[cftime.DatetimeNoLeap(291, 1, 1, 0, 0, 0, 0, 4, 1), cftime.DatetimeNoLeap(301, 1, 1, 0, 0, 0, 0, 0, 1)]


Experiment: cesm1_hist_esm
Bounds
----
[cftime.DatetimeNoLeap(1990, 1, 1, 0, 0, 0, 0, 2, 1), cftime.DatetimeNoLeap(2000, 1, 1, 0, 0, 0, 0, 5, 1)]


Experiment: cesm2_SSP5-8.5
Bounds
----
[cftime.DatetimeNoLeap(2090, 1, 1, 0, 0, 0, 0, 4, 1), cftime.DatetimeNoLeap(2100, 1, 1, 0, 0, 0, 0, 0, 1)]




In [10]:
def compute_global_averages(datasets, integral_units, variable):
    experiments = list(datasets[variable].keys())
    glb_avg = dict()
    new_units = dict()
    for exp in experiments:
        wgts = datasets[variable][exp]['TAREA'].isel(time=0)
        # Note that CESM2 has vertical integrals where CESM1 doesn't, so
        # same variable might have different units in different experiments
        unit_key = 'area'
        dims = ['nlat', 'nlon']
        if 'z_t_150m' in datasets[variable][exp][variable].dims:
            wgts = wgts * datasets[variable][exp]['dz'].isel(time=0, z_t=slice(0,15))
            wgts = wgts.rename({'z_t' : 'z_t_150m'})
            dims.append('z_t_150m')
            unit_key = 'volume'
        elif 'z_t' in datasets[variable][exp][variable].dims:
            wgts = wgts * datasets[variable][exp]['dz'].isel(time=0)
            dims.append('z_t')
            unit_key = 'volume'
        glb_avg[exp] = esmlab.weighted_sum(datasets[variable][exp][variable], dim=dims, weights=wgts).to_dataset(name=variable)
        old_units = units[datasets[variable][exp][variable].attrs['units']]
        new_units[exp] = old_units*integral_units[unit_key][exp]
        glb_avg[exp]
    return glb_avg, new_units

In [11]:
%%time

ann_avg = dict()
new_units = dict()
for variable in all_data:
    glb_avgs, new_units[variable] = compute_global_averages(all_data, integral_units, variable)
    ann_avg[variable] = dict()
    for exp in glb_avgs:
        glb_avgs[exp]['time_bound'] = all_data[variable][exp]['time_bound']
        ann_avg[variable][exp] = esmlab.resample(glb_avgs[exp], freq='ann')

CPU times: user 22.1 s, sys: 1.32 s, total: 23.4 s
Wall time: 43.7 s


## Reduce Data Sets

The following table shows global averages (also averaged over specified time slices)

In [12]:
vars = ['photoC_TOT_zint', 'photoC_diat_zint',
        'POC_FLUX_100m', 'CaCO3_FLUX_100m',
        'diaz_Nfix', 'NOx_FLUX', 'NHy_FLUX', 'DENITRIF',
        'FG_CO2', 'O2']

# Define final units
final_units = dict()
final_units['photoC_TOT_zint'] = 'Pg/year'
final_units['photoC_diat_zint'] = 'Pg/year'
final_units['POC_FLUX_100m'] = 'Pg/year'
final_units['CaCO3_FLUX_100m'] = 'Pg/year'
final_units['diaz_Nfix'] = 'Tg/year'
final_units['NOx_FLUX'] = 'Tg/year'
final_units['NHy_FLUX'] = 'Tg/year'
final_units['DENITRIF'] = 'Tg/year'
final_units['FG_CO2'] = 'Pg/year'
final_units['O2'] = 'uM'
final_units['O2_under_5uM'] = 'm^3'
final_units['O2_under_20uM'] = 'm^3'
final_units['O2_under_60uM'] = 'm^3'
final_units['O2_under_80uM'] = 'm^3'
# Define conversion factors
conversions = dict()

### TODO: RUN CONVERSIONS BY MATT
# Final units are TgN / yr
gN = 14.007 * units['g'] / units['mol']
gC = 12.01 * units['g'] / units['mol']

conversions['photoC_TOT_zint'] = gC
conversions['photoC_diat_zint'] = gC
conversions['POC_FLUX_100m'] = gC
conversions['CaCO3_FLUX_100m'] = gC
conversions['diaz_Nfix'] = gN
conversions['NOx_FLUX'] = gN
conversions['NHy_FLUX'] = gN
conversions['DENITRIF'] = gN
conversions['FG_CO2'] = gC
conversions['O2'] = 1/(total_volume[vol_exp] * integral_units["volume"][vol_exp]) # Want O2 to be average concentration
conversions['O2_under_5uM'] = 1
conversions['O2_under_20uM'] = 1
conversions['O2_under_60uM'] = 1
conversions['O2_under_80uM'] = 1

In [13]:
def get_time_and_ensemble_mean(variable, ann_avg, exp, new_units, conversions, final_units):
    return((ann_avg[variable][exp][variable].mean(['time', 'member_id']).values *
            new_units[variable][exp] *
            conversions[variable]
           ).to(final_units[variable]))

In [14]:
%%time

#FIXME:
#       1. define experiments in cell [6]
#       2. replace this experiments with an assert
experiments = list(ann_avg[vars[0]].keys())
diagnostic_values = dict()
for exp in experiments:
    diagnostic_values[exp] = dict()
    # Compute each value by hand
    print(f'100m POC flux for {exp}')
    diagnostic_values[exp]['Sinking POC at 100 m (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('POC_FLUX_100m', ann_avg, exp, new_units, conversions, final_units)

    print(f'100m CaCO3 flux for {exp}')
    diagnostic_values[exp]['Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('CaCO3_FLUX_100m', ann_avg, exp, new_units, conversions, final_units)

    print(f'100m rain rate for {exp}')
    diagnostic_values[exp]['Rain ratio (CaCO$_3$/POC) at 100 m'] = diagnostic_values[exp]['Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)'] / \
                                                                   diagnostic_values[exp]['Sinking POC at 100 m (PgC yr$^{-1}$)']
    
    print(f'Net primary production for {exp}')
    diagnostic_values[exp]['Net primary production (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('photoC_TOT_zint', ann_avg, exp, new_units, conversions, final_units)

    print(f'Primary production from diatoms for {exp}')
    diagnostic_values[exp]['Diatom primary production (%)'] = get_time_and_ensemble_mean('photoC_diat_zint', ann_avg, exp, new_units, conversions, final_units) / \
                                                              diagnostic_values[exp]['Net primary production (PgC yr$^{-1}$)'] * \
                                                              100

    print(f'Computing Nfixation for {exp}')
    diagnostic_values[exp]['Nitrogen fixation (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('diaz_Nfix', ann_avg, exp, new_units, conversions, final_units)
    
    print(f'Computing Ndep for {exp}')
    diagnostic_values[exp]['Nitrogen deposition (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('NOx_FLUX', ann_avg, exp, new_units, conversions, final_units) + \
                                                                    get_time_and_ensemble_mean('NHy_FLUX', ann_avg, exp, new_units, conversions, final_units)
    print(f'Computing Denitrif for {exp}')
    diagnostic_values[exp]['Denitrification (TgN yr$^{-1}$)'] = get_time_and_ensemble_mean('DENITRIF', ann_avg, exp, new_units, conversions, final_units)
    
    print(f'Computing Nitrogen Cycle imbalance for {exp}')
    diagnostic_values[exp]['N cycle imbalance* (TgN yr$^{-1}$)'] = diagnostic_values[exp]['Nitrogen deposition (TgN yr$^{-1}$)'] + \
                                                                   diagnostic_values[exp]['Nitrogen fixation (TgN yr$^{-1}$)'] - \
                                                                   diagnostic_values[exp]['Denitrification (TgN yr$^{-1}$)']

    print(f'Air-Sea CO2 Flux for {exp}')
    diagnostic_values[exp]['Air–sea CO2 flux (PgC yr$^{-1}$)'] = get_time_and_ensemble_mean('FG_CO2', ann_avg, exp, new_units, conversions, final_units)
        
    print(f'Oxygen for {exp}')
    diagnostic_values[exp]['Mean ocean oxygen ($\mu$M)'] = get_time_and_ensemble_mean('O2', ann_avg, exp, new_units, conversions, final_units)
        
    print(f'Computing OMZ volume for {exp}')
    diagnostic_values[exp]['OMZ volume (10$^{15}$ m$^3$; <20 $\mu$M)'] = get_time_and_ensemble_mean('O2_under_20uM', ann_avg, exp, new_units, conversions, final_units) * \
                                                                         (10**-15)

    for thres in [5, 60, 80]:
        print(f'Computing volume where O2 < {thres} uM for {exp}')
        diagnostic_values[exp][f'Volume (10$^1$$^5$ m$^3$) where O$_2$ <{thres} $\mu$M)'] = get_time_and_ensemble_mean(f'O2_under_{thres}uM', ann_avg, exp, new_units, conversions, final_units) * \
                                                                                           (10**-15)

    if exp != experiments[-1]:
        print('\n----\n')


100m POC flux for cesm1_PI
100m CaCO3 flux for cesm1_PI
100m rain rate for cesm1_PI
Net primary production for cesm1_PI
Primary production from diatoms for cesm1_PI
Computing Nfixation for cesm1_PI
Computing Ndep for cesm1_PI
Computing Denitrif for cesm1_PI
Computing Nitrogen Cycle imbalance for cesm1_PI
Air-Sea CO2 Flux for cesm1_PI
Oxygen for cesm1_PI
Computing OMZ volume for cesm1_PI
Computing volume where O2 < 5 uM for cesm1_PI
Computing volume where O2 < 60 uM for cesm1_PI
Computing volume where O2 < 80 uM for cesm1_PI

----

100m POC flux for cesm1_hist_esm
100m CaCO3 flux for cesm1_hist_esm
100m rain rate for cesm1_hist_esm
Net primary production for cesm1_hist_esm
Primary production from diatoms for cesm1_hist_esm
Computing Nfixation for cesm1_hist_esm
Computing Ndep for cesm1_hist_esm
Computing Denitrif for cesm1_hist_esm
Computing Nitrogen Cycle imbalance for cesm1_hist_esm
Air-Sea CO2 Flux for cesm1_hist_esm
Oxygen for cesm1_hist_esm
Computing OMZ volume for cesm1_hist_esm
C

In [15]:
# Fill a dict with (data, units) tuple [unit conversion comes later]
table_dict = dict()
diagnostic_columns = ['Net primary production (PgC yr$^{-1}$)',
                      'Sinking POC at 100 m (PgC yr$^{-1}$)',
                      'Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$)',
                      'Rain ratio (CaCO$_3$/POC) at 100 m',
                      'Nitrogen fixation (TgN yr$^{-1}$)',
                      'Nitrogen deposition (TgN yr$^{-1}$)',
                      'Denitrification (TgN yr$^{-1}$)',
                      'N cycle imbalance* (TgN yr$^{-1}$)',
                      'Air–sea CO2 flux (PgC yr$^{-1}$)',
                      'Diatom primary production (%)',
                      'Mean ocean oxygen ($\mu$M)',
                      'OMZ volume (10$^{15}$ m$^3$; <20 $\mu$M)',
                      'Volume (10$^1$$^5$ m$^3$) where O$_2$ <5 $\mu$M)',
                      'Volume (10$^1$$^5$ m$^3$) where O$_2$ <60 $\mu$M)',
                      'Volume (10$^1$$^5$ m$^3$) where O$_2$ <80 $\mu$M)'
                     ]
experiment_longnames={'cesm1_hist' : '1990s (CESM1)',
                      'cesm1_hist_esm' : '1990s (CESM1, BPRP)',
                      'cesm1_PI' : 'preindustrial (CESM1)',
                      'cesm2_hist' : '1990s (CESM2)',
                      'cesm2_SSP5-8.5' : 'RCP85 2090s (CESM2)'}

table_dict['Diagnostic'] = []
for variable in diagnostic_columns:
    table_dict['Diagnostic'].append(variable)
    for exp in experiments:
        if experiment_longnames[exp] not in table_dict:
            table_dict[experiment_longnames[exp]] = []
        try:
            table_dict[experiment_longnames[exp]].append(diagnostic_values[exp][variable].magnitude)
        except:
            table_dict[experiment_longnames[exp]].append('-')

pd.DataFrame(table_dict)

,Diagnostic,preindustrial (CESM1),"1990s (CESM1, BPRP)",RCP85 2090s (CESM2)
0,Net primary production (PgC yr$^{-1}$),1.107053,1.106861,50.276790
1,Sinking POC at 100 m (PgC yr$^{-1}$),8.084837,8.074632,6.742707
2,Sinking CaCO$_3$ at 100 m (PgC yr$^{-1}$),0.754636,0.752547,0.814445
3,Rain ratio (CaCO$_3$/POC) at 100 m,0.093340,0.093199,0.120789
4,Nitrogen fixation (TgN yr$^{-1}$),174.722114,173.900438,287.298661
5,Nitrogen deposition (TgN yr$^{-1}$),6.709413,30.055928,38.829656
6,Denitrification (TgN yr$^{-1}$),189.937453,193.491725,265.682128
7,N cycle imbalance* (TgN yr$^{-1}$),-8.505926,10.464641,60.446188
8,Air–sea CO2 flux (PgC yr$^{-1}$),0.004003,2.189154,5.324415
9,Diatom primary production (%),1758.081176,1770.291946,30.709419
